In [261]:
import glob
import time
import numpy
import crowdsource
import regions
import numpy as np
from functools import cache
from astropy.convolution import convolve, Gaussian2DKernel
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.visualization import simple_norm
from astropy.modeling.fitting import LevMarLSQFitter
from astropy import wcs
from astropy import table
from astropy import stats
from astropy import units as u
from astropy.nddata import NDData
from astropy.io import fits
from scipy import ndimage
import requests
import requests.exceptions
import urllib3
import urllib3.exceptions
from photutils.detection import DAOStarFinder, IRAFStarFinder
from photutils.psf import IntegratedGaussianPRF, extract_stars, EPSFStars, EPSFModel
try:
    # version >=1.7.0, doesn't work: the PSF is broken
    from photutils.psf import PSFPhotometry, IterativePSFPhotometry, SourceGrouper
except:
    # version 1.6.0, which works
    from photutils.psf import BasicPSFPhotometry as PSFPhotometry, IterativelySubtractedPSFPhotometry as IterativePSFPhotometry, DAOGroup as SourceGrouper
try:
    from photutils.background import MMMBackground, MADStdBackgroundRMS, MedianBackground, Background2D, LocalBackground
except:
    from photutils.background import MMMBackground, MADStdBackgroundRMS, MedianBackground, Background2D
    from photutils.background import MMMBackground as LocalBackground

from photutils.psf import EPSFBuilder
from photutils.psf import extract_stars

import warnings
from astropy.utils.exceptions import AstropyWarning, AstropyDeprecationWarning
warnings.simplefilter('ignore', category=AstropyWarning)
warnings.simplefilter('ignore', category=AstropyDeprecationWarning)

from crowdsource import crowdsource_base
from crowdsource.crowdsource_base import fit_im, psfmod

from astroquery.svo_fps import SvoFps

import pylab as pl
pl.rcParams['figure.facecolor'] = 'w'
pl.rcParams['image.origin'] = 'lower'

import os
print("Importing webbpsf", flush=True)
import webbpsf
from webbpsf.utils import to_griddedpsfmodel
import datetime

import scipy
import scipy.ndimage
import scipy.spatial
from scipy.spatial import KDTree


Importing webbpsf


In [262]:
import importlib as imp
imp.reload(crowdsource)
from crowdsource import crowdsource_base
imp.reload(crowdsource.crowdsource_base)
from crowdsource.crowdsource_base import fit_im, psfmod
from webbpsf.utils import to_griddedpsfmodel

In [263]:
#filt = filtername = 'f410m'
filt = filtername = 'f444w'
im1 = fits.open(f'{basepath}/F444W/pipeline/jw01182004001_04101_00010_nrcblong_destreak_o004_crf.fits')
obsdate = im1[0].header['DATE-OBS']

In [ ]:
has_downloaded = False
ntries = 0
while not has_downloaded:
    try:
        nrc = webbpsf.NIRCam()
        nrc.load_wss_opd_by_date(f'{obsdate}T00:00:00')
        nrc.filter = filt
        if module in ('nrca', 'nrcb'):
            if 'F4' in filt.upper():
                nrc.detector = f'{module.upper()}5' # I think NRCA5 must be the "long" detector?
            else:
                nrc.detector = f'{module.upper()}1' #TODO: figure out a way to use all 4?
            grid = nrc.psf_grid(num_psfs=16, all_detectors=False, verbose=True, save=True)
        else:
            grid = nrc.psf_grid(num_psfs=16, all_detectors=True, verbose=True, save=True)
        has_downloaded = True
    except (urllib3.exceptions.ReadTimeoutError, requests.exceptions.ReadTimeout, requests.HTTPError) as ex:
        print(f"Failed to build PSF: {ex}", flush=True)
    except Exception as ex:
        print(ex, flush=True)
        if ntries > 10:
            # avoid infinite loops
            raise ValueError("Failed to download PSF, probably because of an error listed above")
        else:
            continue

In [ ]:
basepath = '/blue/adamginsburg/adamginsburg/jwst/brick/'

In [ ]:
ww = wcs.WCS(im1[1].header)

module = 'nrcb'
data = im1['SCI'].data[:400,:400]
err = im1['ERR'].data[:400,:400]
if 'WHT' in im1:
    wht = im1['WHT'].data[:400,:400]
else:
    wht = 1
dq = im1['DQ'].data[:400, :400]

In [ ]:
im1.info()

In [ ]:
import sys
sys.path.append(f'{basepath}/analysis')
import crowdsource_catalogs_long
import importlib
importlib.reload(crowdsource_catalogs_long)
from crowdsource_catalogs_long import WrappedPSFModel, get_uncertainty

In [ ]:
psf_model = WrappedPSFModel(grid)

In [ ]:
pl.imshow(im1['DQ'].data[5:400,5:400], cmap='jet');
pl.colorbar();

# Start tests

In [ ]:
dq, weight, bad = get_uncertainty(err, data, wht=wht, dq=dq)

In [ ]:
data.shape, data.size, mask.sum(), bad.sum()

In [ ]:
from scipy.spatial import KDTree

# crowdsource

In [ ]:
results  = fit_im(np.nan_to_num(data), psf_model, weight=weight,
                  dq=dq,
                  nskyx=0, nskyy=0, refit_psf=False, verbose=True)
stars, modsky, skymsky, psf = results
residual = data - modsky
model = modsky
result = stars = Table(stars)
stars['x'], stars['y'] = stars['y'], stars['x']

In [ ]:
len(stars)

In [ ]:
Table(stars[:3])

In [ ]:
pl.scatter(result['x'], result['y'], s=1)

In [ ]:
dist, ind = KDTree(np.array([result['x'], result['y']]).T).query(np.array([result['x'], result['y']]).T, 2) 
close_neighbor = ind[:,1][dist[:,1] < 1]

len(close_neighbor), len(stars)

### Default

### Histograms

In [ ]:
pl.hist(result['flux'], bins=np.logspace(0,5), histtype='step')
#pl.hist(result_g['flux_init'], bins=np.logspace(0,5), histtype='step')
pl.xscale('log');

In [ ]:
pl.hist(result['flux'][close_neighbor], bins=np.logspace(0,5), histtype='step', label='default')
pl.xscale('log');
pl.legend(loc='best');

In [ ]:
    np.unique(dq)

In [ ]:
good = (stars['qf'] > 0.9) & (stars['fracflux'] > 0.75)

In [ ]:
pl.figure(figsize=(16,10))
pl.subplot(2,3,1).imshow(data[:128,:128], norm=simple_norm(data[:256,:256], stretch='log', max_percent=99.95, min_percent=0.5), cmap='gray')
pl.xticks([]); pl.yticks([]); pl.title("Data")
pl.subplot(2,3,2).imshow(modsky[:128,:128], norm=simple_norm(modsky[:256,:256], stretch='log', max_percent=99.95, min_percent=0.5), cmap='gray')
pl.xticks([]); pl.yticks([]); pl.title("fit_im model+sky")
pl.subplot(2,3,4).imshow((data-modsky)[:128,:128], norm=simple_norm((data-modsky)[:256,:256], stretch='asinh',
                                                                    max_percent=99.5, min_percent=0.5), cmap='gray')
pl.xticks([]); pl.yticks([]); pl.title("data-modsky")
pl.subplot(2,3,5).imshow(data[:128,:128], norm=simple_norm(data[:256,:256], stretch='log', max_percent=99.95, min_percent=0.5), cmap='gray')
pl.subplot(2,3,5).scatter(stars['x'], stars['y'], marker='x', color='b', s=8, linewidth=0.5)
pl.subplot(2,3,5).scatter(stars['x'][good], stars['y'][good], marker='x', color='r', s=6, linewidth=0.5)
pl.xticks([]); pl.yticks([]); pl.title("Data with stars");
pl.axis([0,128,0,128])
pl.subplot(2,3,3).imshow((dq)[:128,:128], norm=simple_norm((dq), stretch='log', min_cut=0, max_cut=5310465), cmap='jet')
pl.subplot(2,3,3).scatter(stars['x'], stars['y'], marker='x', color='w', s=4, linewidth=0.5)
pl.axis([0,128,0,128])
pl.xticks([]); pl.yticks([]); pl.title("DQ");
pl.subplot(2,3,6).imshow(weight[:128,:128], norm=simple_norm(weight[:128,:128], stretch='asinh', min_percent=5, max_percent=99.95), cmap='gray')
#pl.subplot(2,3,6).scatter(stars['x'], stars['y'], marker='x', color='r', s=4, linewidth=0.5)
pl.axis([0,128,0,128])
pl.xticks([]); pl.yticks([]); pl.title("weight");
pl.suptitle("");
pl.tight_layout();

In [ ]:
pl.figure(figsize=(16,10))
pl.subplot(2,3,1).imshow(data[:128,-128:], norm=simple_norm(data[:256,:256], stretch='log', max_percent=99.95, min_percent=0.5), cmap='gray')
pl.xticks([]); pl.yticks([]); pl.title("Data")
pl.subplot(2,3,2).imshow(modsky[:128,-128:], norm=simple_norm(modsky[:256,:256], stretch='log', max_percent=99.95, min_percent=0.5), cmap='gray')
pl.xticks([]); pl.yticks([]); pl.title("fit_im model+sky")
pl.subplot(2,3,4).imshow((data-modsky)[:128,-128:], norm=simple_norm((data-modsky)[:256,:256], stretch='asinh', max_percent=99.5, min_percent=0.5), cmap='gray')
pl.xticks([]); pl.yticks([]); pl.title("data-modsky")
pl.subplot(2,3,5).imshow(data[:128,-128:], norm=simple_norm(data[:256,:256], stretch='log', max_percent=99.95, min_percent=0.5), cmap='gray')
pl.subplot(2,3,5).scatter(stars['x']-(400-128), stars['y'], marker='x', color='b', s=8, linewidth=0.5)
pl.subplot(2,3,5).scatter(stars['x'][good]-(400-128), stars['y'][good], marker='x', color='r', s=6, linewidth=0.5)
pl.xticks([]); pl.yticks([]); pl.title("Data with stars");
pl.axis([0,128,0,128])
pl.subplot(2,3,3).imshow((dq)[:128,-128:], norm=simple_norm((dq), stretch='log', min_cut=0, max_cut=5310465), cmap='jet')
pl.subplot(2,3,3).scatter(stars['x']-(400-128), stars['y'], marker='x', color='w', s=4, linewidth=0.5)
pl.axis([0,128,0,128])
pl.xticks([]); pl.yticks([]); pl.title("DQ");
pl.subplot(2,3,6).imshow(weight[:128,-128:], norm=simple_norm(weight[:128,-128:], stretch='asinh', min_percent=5, max_percent=99.95), cmap='gray')
#pl.subplot(2,3,6).scatter(stars['x'], stars['y'], marker='x', color='r', s=4, linewidth=0.5)
pl.axis([0,128,0,128])
pl.xticks([]); pl.yticks([]); pl.title("weight");
pl.suptitle("");
pl.tight_layout();

In [ ]:
pl.figure(figsize=(16,10))
pl.subplot(2,3,1).imshow(data[:128,:128], norm=simple_norm(data[:256,:256], stretch='log', max_percent=99.95, min_percent=0.5), cmap='gray')
pl.xticks([]); pl.yticks([]); pl.title("Data")
pl.subplot(2,3,2).imshow(modsky[:128,:128], norm=simple_norm(modsky[:256,:256], stretch='log', max_percent=99.95, min_percent=0.5), cmap='gray')
pl.xticks([]); pl.yticks([]); pl.title("fit_im model+sky")
pl.subplot(2,3,4).imshow((data-modsky)[:128,:128], norm=simple_norm((data-modsky)[:256,:256], stretch='asinh', max_percent=99.5, min_percent=0.5), cmap='gray')
pl.xticks([]); pl.yticks([]); pl.title("data-modsky")
pl.subplot(2,3,5).imshow(data[:128,:128], norm=simple_norm(data[:256,:256], stretch='log', max_percent=99.95, min_percent=0.5), cmap='gray')
pl.subplot(2,3,5).scatter(stars['x'], stars['y'], marker='x', color='b', s=8, linewidth=0.5)
pl.subplot(2,3,5).scatter(stars['x'][good], stars['y'][good], marker='x', color='r', s=4, linewidth=0.5)
pl.xticks([]); pl.yticks([]); pl.title("Data with stars");
pl.axis([0,128,0,128])
pl.subplot(2,3,3).imshow((dq)[:128,:128], norm=simple_norm((dq), stretch='log', min_cut=0, max_cut=5310465), cmap='jet')
pl.subplot(2,3,3).scatter(stars['x'], stars['y'], marker='x', color='w', s=4, linewidth=0.5)
pl.axis([0,128,0,128])
pl.xticks([]); pl.yticks([]); pl.title("DQ");
pl.subplot(2,3,6).imshow(weight[:128,:128], norm=simple_norm(weight[:128,:128], stretch='asinh', min_percent=5, max_percent=99.95), cmap='gray')
#pl.subplot(2,3,6).scatter(stars['x'], stars['y'], marker='x', color='r', s=4, linewidth=0.5)
pl.axis([0,128,0,128])
pl.xticks([]); pl.yticks([]); pl.title("weight");
pl.suptitle("");
pl.tight_layout();

In [ ]:
pl.figure(dpi=200)
pl.imshow(data, norm=simple_norm(data, min_cut=0, max_cut=20, stretch='asinh'))
pl.colorbar();

In [ ]:
pl.figure(dpi=200)
pl.imshow(residual, norm=simple_norm(residual, min_cut=-2, max_cut=20, stretch='log'), cmap='gray')
pl.colorbar();

In [ ]:
pl.figure(dpi=200)
pl.imshow(model, norm=simple_norm(model, min_percent=1, max_percent=99, stretch='log'))
pl.colorbar();

In [ ]:
xl,xu,yl,yu = 100,300,200,400
slc = slice(yl,yu), slice(xl,xu)

In [ ]:
pl.figure(dpi=200)
pl.imshow(model[slc], norm=simple_norm(model, min_percent=1, max_percent=99, stretch='log'))
pl.colorbar();
pl.scatter(result['x']-xl, result['y']-yl, marker='x', color='r', linewidths=0.5, s=8)
pl.axis([0,200,0,200])

In [ ]:
pl.figure(dpi=200)
pl.imshow(data[slc], norm=simple_norm(data, min_percent=1, max_percent=99, stretch='log'))
pl.colorbar();
pl.scatter(result['x']-xl, result['y']-yl, marker='x', color='r', linewidths=0.5, s=8)
pl.axis([0,200,0,200]);

In [ ]:
pl.figure(dpi=200)
pl.imshow(residual[slc], norm=simple_norm(residual, min_cut=-1, max_cut=25, stretch='asinh'))
pl.colorbar();
pl.scatter(result['x']-xl, result['y']-yl, marker='x', color='r', linewidths=0.5, s=8)
pl.axis([0,200,0,200]);

# Residual zoom

In [ ]:
import sys
sys.path.append(f'{basepath}/analysis')
import plot_tools
import importlib as imp
imp.reload(plot_tools)
from plot_tools import diagnostic_stamps_by_mag_crowdsource
def diagnostic_stamps_by_mag(*args, **kwargs):
    return diagnostic_stamps_by_mag_crowdsource(*args,
                                                filtername=filtername,
                                                pixel_area=ww.celestial.proj_plane_pixel_area(), data=data, **kwargs)

In [ ]:
diagnostic_stamps_by_mag(result, residual, min_qf=0.95, min_fracflux=0.75, max_mag=20, min_mag=16)

In [ ]:
diagnostic_stamps_by_mag(result, residual, min_qf=0.95, min_fracflux=0.75, max_mag=16, min_mag=12)

In [ ]:
diagnostic_stamps_by_mag(result, residual, min_qf=0.95, min_fracflux=0.75, ind_offset=1, max_mag=16, min_mag=12)

In [ ]:
diagnostic_stamps_by_mag(result, residual, min_qf=0.95, min_fracflux=0.75, ind_offset=1, max_mag=20, min_mag=16)

In [ ]:
diagnostic_stamps_by_mag(result, residual, min_qf=0.95, min_fracflux=0.75, ind_offset=2)

In [ ]:
diagnostic_stamps_by_mag(result, residual)

In [ ]:
diagnostic_stamps_by_mag(result, residual, ind_offset=1)

In [ ]:
diagnostic_stamps_by_mag(result, residual, ind_offset=2)

In [ ]:
diagnostic_stamps_by_mag(result, residual, ind_offset=3)